# main function: run model but able to get the limit of the cell and how it finally die
1. run model based on last solution
2. try every run model only
3. if fail at some point, started the 'conserved' mode, run cycle by cycle (inplace=true), try smarter
4. upon limited, try until the final step; further, try upon the final seconds

In [1]:
import pybamm as pb;import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os;#import imageio
from scipy.io import savemat,loadmat;from pybamm import constants,exp;import matplotlib as mpl; fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
for k in range(0,1):
    mpl.rcParams["axes.labelsize"] = fs
    mpl.rcParams["axes.titlesize"] = fs
    mpl.rcParams["xtick.labelsize"] =  fs
    mpl.rcParams["ytick.labelsize"] =  fs
    mpl.rcParams["legend.fontsize"] =  fs
    mpl.rcParams['font.sans-serif'] = ['Times New Roman']
    mpl.rcParams['font.serif'] = ['Times New Roman']
    mpl.rcParams['axes.unicode_minus'] = False
import openpyxl
import traceback
import multiprocessing

In [4]:
V_max = 4.2; V_min = 2.5; 
total_cycle = 8; save_at_cycles= 3; loop_num = int(total_cycle/save_at_cycles);
exp_text = [ (f"Discharge at 1C until {V_min} V",  
        "Rest for 1 hours (5 minute period)",  
        f"Charge at 1C until {V_max} V (5 minute period)" ) ]
Experiment = pb.Experiment( exp_text * save_at_cycles ) 

Chemistry=pb.parameter_sets.OKane2022 
Para_0=pb.ParameterValues(chemistry=Chemistry) 
Para_0.update({"Upper voltage cut-off [V]": 4.21})
Para_0.update({"Lower voltage cut-off [V]": 2.49})
Para_0.update({"Inner SEI reaction proportion": 0.5})
Para_0.update({"Ratio of lithium moles to SEI moles": 1})
Para_0.update({"Initial inner SEI thickness [m]": 2.5E-9})
Para_0.update({"Initial outer SEI thickness [m]": 2.5E-9})
Para_0.update({"SEI growth activation energy [J.mol-1]": 38000})
Para_0.update(
    {"Outer SEI solvent diffusivity [m2.s-1]": 1e-17})

In [5]:
Model_0 = pb.lithium_ion.DFN(options={
    "SEI":"solvent-diffusion limited",   
    "SEI film resistance":"distributed",          
    "SEI porosity change":"true",      } ) #
Sim_0    = pb.Simulation(
    Model_0,        experiment = Experiment,
    parameter_values = Para_0,
    solver = pb.CasadiSolver(),) #mode="safe"
Sol_0    = Sim_0.solve(save_at_cycles=save_at_cycles,)

In [6]:
label = ["0",] 
output_variables3 = [
    "Electrolyte potential [V]",
    "Porosity",
    "Positive electrode potential [V]",
    "Negative electrode potential [V]",
    "Positive current collector potential [V]", 
    "Terminal voltage [V]", 
]
quick_plot = pb.QuickPlot(
    [Sol_0,], output_variables3,label,variable_limits='fixed',
    time_unit='hours',
    spatial_unit='mm',     #  (“m”, “mm”, or “um”)
    n_rows=2) #figsize = (18,12),
quick_plot.dynamic_plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=5.714888808360271, step=0.05714888808360272)…

In [ ]:
exp_text = [ (f"Discharge at 1C until {V_min} V",  
        "Rest for 1 hours (5 minute period)",  
        f"Charge at 1C until {V_max} V (5 minute period)" ) ]
Experiment = pb.Experiment( exp_text * save_at_cycles ) 

In [ ]:
# Run model based on last solution, and try again:
k=0; old_sol = Sol_0; old_model = Model_0; sol_all =[];
while k < loop_num-1:  
    new_model = old_model.set_initial_conditions_from(old_sol, inplace=False)
    new_sim    = pb.Simulation(
        new_model,        experiment = Experiment,
        parameter_values = Para_0,
        solver = pb.CasadiSolver(),) #mode="safe"
    try:
        new_sol= new_sim.solve(save_at_cycles=save_at_cycles,)
        print(new_sol.cycles[-1].steps[-1]["Ternimal voltage [V]"].entries[-1])
    except: # note that old_sol and old_model are both here now
        # if fail, need to run another round of try, how to run conservatively?
        print(f'Fail in the No.{k} loop, now try to run conservatively') # f"Charge at 1C until {V_max} 
        # run the model agan but with one less cycle
        conser_model = old_model.set_initial_conditions_from(old_sol, inplace=False)
        conser_Exp = pb.Experiment( exp_text * save_at_cycles-1 ) 
        conser_sim    = pb.Simulation(
            conser_model,        experiment = conser_Exp,
            parameter_values = Para_0,
            solver = pb.CasadiSolver(),) #mode="safe"
        conser_sol= conser_sim.solve()
    else:     # if succeed, continue
        del old_sol,old_model
        old_sol = new_sol.copy(); old_model = new_model.copy();
        sol_all.appeng(old_sol); 
        del new_sol,new_model
    
